# Proposal

## Vision

My final project utilizes the text categorization potential of spaCy. spaCy is a Python library for NLP. "spaCy is designed specifically for production use and helps you build applications that process and “understand” large volumes of text. It can be used to build information extraction or natural language understanding systems, or to pre-process text for deep learning." (https://spacy.io/usage/spacy-101)

While we have done deep learning with NLP in class before, we haven't really explored any specialized tools such as spaCy, and I think it's good to show what deep learning can be like in the context of a domain-specific tool like this. You can add a model, in this case a TextCategorizer, to the spaCy pipeline and then save the model to a file so that it is easy to open up that model again and apply it to new data whenever you want. So spaCy's TextCategorizer is particularly useful if you already know you want to work with spaCy (e.g., for its tokenization, sentence segmentation, part of speech tagging, and/or many other features), since you can just do text categorization along with all the other stuff you're doing to the text.

I also have an interest in this project specifically because another final project I am working on (for Data 303) uses spaCy and if we get far enough into that project we will want to put sentences (or collections of sentences) into categories, so it could be useful to have more understanding and experience of how spaCy's TextCategorizer works.

## Background

Describe the technologies you’re using and why you chose them. Include citations of the work on which you’ve based your system. Include citations for the technologies we’ve covered in class (no description is required for these) and the new technologies you’ve experimented with (include descriptions of these).

### spaCy overview
"When you call nlp on a text, spaCy first tokenizes the text to produce a Doc object. The Doc is then processed in several different steps – this is also referred to as the processing pipeline. The pipeline used by the default models consists of a tagger, a parser and an entity recognizer. Each pipeline component returns the processed Doc, which is then passed on to the next component." First is the tokenizer, which segments text into tokens. Some other pipeline components are the tagger (part of speech tags), parser (dependency parsing, which is the default for sentence segmentation), textcat (text categorizer), and any custom components the user decides to add. 

"The processing pipeline always depends on the statistical model and its capabilities. For example, a pipeline can only include an entity recognizer component if the model includes data to make predictions of entity labels." (https://spacy.io/usage/processing-pipelines#_title)

### Training spaCy's Statistical Models
https://spacy.io/usage/training

"spaCy’s models are statistical and every “decision” they make – for example, which part-of-speech tag to assign, or whether a word is a named entity – is a prediction. This prediction is based on the examples the model has seen during training." To train a model, you need to provide labeled training data for the model to learn from. (In spaCy you can choose to update an existing model so that you don't have to start from scratch with training. My project does not use this feature, since there was not an existing TextCategorizer component that I wanted to build on.)

In the simple training style, you "pass raw texts and dictionaries of annotations to nlp.update. The dictionaries can have the keys entities, heads, deps, tags and cats."

### Optimization
https://spacy.io/usage/training

There are lots of conflicting “recipes” for training deep neural networks at the moment. The cutting-edge models take a very long time to train, so most researchers can’t run enough experiments to figure out what’s really going on. For what it’s worth, here’s a recipe that seems to work well on a lot of NLP problems:

1. Initialize with batch size 1, and compound to a maximum determined by your data size and problem type.
2. Use Adam solver with fixed learning rate.
3. Use averaged parameters
4. Use L2 regularization.
5. Clip gradients by L2 norm to 1.
6. On small data sizes, start at a high dropout rate, with linear decay.

#### Batch size
We learned about batch size in https://developers.google.com/machine-learning/crash-course/reducing-loss/video-lecture, but we didn't learn about compounding batch size. 

"In training the various spaCy models, we haven’t found much advantage from decaying the learning rate – but starting with a low batch size has definitely helped." (see the example at https://spacy.io/usage/training#tips-batch-size) "This will set the batch size to start at 1, and increase each batch until it reaches a maximum size. The tagger, parser and entity recognizer all take whole sentences as input, so they’re learning a lot of labels in a single example. You therefore need smaller batches for them. The batch size for the text categorizer should be somewhat larger, especially if your documents are long."

#### Regularization and gradient clipping
We learned about L2 regularization in https://developers.google.com/machine-learning/crash-course/descending-into-ml/video-lecture and https://developers.google.com/machine-learning/crash-course/regularization-for-simplicity/video-lecture. We used Adam solver in https://colab.research.google.com/notebooks/mlcc/improving_neural_net_performance.ipynb.

"By default spaCy uses the Adam solver, with default settings (`learn_rate=0.001, beta1=0.9, beta2=0.999`). Some researchers have said they found these settings terrible on their problems – but they’ve always performed very well in training spaCy’s models, in combination with the rest of our recipe. You can change these settings directly, by modifying the corresponding attributes on the optimizer object. You can also set environment variables, to adjust the defaults."

"There are two other key hyper-parameters of the solver: L2 regularization, and gradient clipping (`max_grad_norm`). Gradient clipping is a hack that’s not discussed often, but everybody seems to be using. It’s quite important in helping to ensure the network doesn’t diverge, which is a fancy way of saying “fall over during training”. The effect is sort of similar to setting the learning rate low. It can also compensate for a large batch size (this is a good example of how the choices of all these hyper-parameters intersect)."

#### Dropout rate
We learned about dropout in https://developers.google.com/machine-learning/crash-course/training-neural-networks/video-lecture, but we didn't learn about decay of dropout rate.

"For small datasets, it’s useful to set a high dropout rate at first, and decay it down towards a more reasonable value. This helps avoid the network immediately overfitting, while still encouraging it to learn some of the more interesting things in your data. spaCy comes with a `decaying` utility function to facilitate this."

"All of the models currently use Bernoulli dropout, for no particularly principled reason – we just haven’t experimented with another scheme like Gaussian dropout yet."

#### Parameter averaging
We did not talk about this in class.

"The trick is to store the moving average of the weights during training. We don’t optimize this average – we just track it. Then when we want to actually use the model, we use the averages, not the most recent value. In spaCy (and Thinc) this is done by using a context manager, use_params, to temporarily replace the weights"

"The context manager is handy because you naturally want to evaluate and save the model at various points during training (e.g. after each epoch). After evaluating and saving, the context manager will exit and the weights will be restored, so you resume training from the most recent value, rather than the average. By evaluating the model after each epoch, you can remove one hyper-parameter from consideration (the number of epochs). Having one less magic number to guess is extremely nice – so having the averaging under a context manager is very convenient."

I'm not completely sure how this works. As I understand it, the optimization uses the latest weights, but the evaluation (and what you would want to save to disk) uses the average of the weights. I think maybe part of the idea is you could do evaluation (with parameter averaging after each epoch) so you can do early stopping when the model seems to be doing good enough and/or starts overfitting.

### TextCategorizer configuration parameters
The documentation is at https://spacy.io/api/textcategorizer.

- `exclusive_classes`: "Make categories mutually exclusive. Defaults to `False`."
  - You would use this if you are doing multiclass classification or if you are doing binary classification where you add both categories (the example code for TextCategorizer predicts positive and negative categories from IMDB reviews, and it adds both categories https://github.com/explosion/spaCy/blob/master/examples/training/train_textcat.py ). We learned about classification in https://developers.google.com/machine-learning/crash-course/classification/video-lecture and multiclass classification in https://developers.google.com/machine-learning/crash-course/multi-class-neural-networks/video-lecture
- `architecture`: "Model architecture to use, see architechtures for details. Defaults to "**ensemble**"."
- `"ensemble"`: "Stacked ensemble of a bag-of-words model and a neural network model. The neural network uses a CNN with mean pooling and attention. The "ngram_size" and "attr" arguments can be used to configure the feature extraction for the bag-of-words model."
  - I don't understand fully how this works and it didn't seem necessary for my project, so I didn't use it.
- `"simple_cnn"`: "A neural network model where token vectors are calculated using a CNN. The vectors are mean pooled and used as features in a feed-forward network. This architecture is usually less accurate than the ensemble, but runs faster."
  - We learned about neural networks in https://developers.google.com/machine-learning/crash-course/introduction-to-neural-networks/video-lecture. We learned about convolutional neural networks in https://developers.google.com/machine-learning/practica/image-classification/convolutional-neural-networks.)
- `"bow"`: "An ngram "bag-of-words" model. This architecture should run much faster than the others, but may not be as accurate, especially if texts are short. The features extracted can be controlled using the keyword arguments ngram_size and attr. For instance, ngram_size=3 and attr="lower" would give lower-cased unigram, trigram and bigram features. 2, 3 or 4 are usually good choices of ngram size."
  - I don't think we talked about ngrams in class. In this context, an ngram is an ordered collection of tokens (for example, "the" is a unigram (since it is only one token), "the food" is a bigram, "the food was" is a trigram, and "the food was good" is a 4-gram. In spaCy's implementation, if a higher ngram is extracted then the lower ngrams are also extracted (e.g., if you are using 4-grams you are also using trigrams, bigrams, and unigrams). You can see this in spacy's implementation of `extract_ngrams` at line 134 in https://github.com/explosion/spaCy/blob/master/spacy/_ml.py. For more information about ngrams, you can read https://en.wikipedia.org/wiki/N-gram.
  - The bag-of-words model is a linear model (source code at https://github.com/explosion/spaCy/blob/master/spacy/_ml.py in `build_bow_text_classifier`) with the "words" (by which I mean ngrams) as features. We did a bag-of-words model in https://colab.research.google.com/notebooks/mlcc/intro_to_sparse_data_and_embeddings.ipynb. That only had unigrams, but this architecture allows higher ngrams. The default is ngram size 1 (so only unigrams).
  - The bag-of-words model is described here: https://en.wikipedia.org/wiki/Bag-of-words_model
- `token_vector_width`: Length of token vectors. Defaults to the enviornment variable, which has the default value of `128`. If the environment variable is not provided, a default of `96` is used. Used by `"simple_cnn"` architecture. (Missing from documentation but can be found in source code at https://github.com/explosion/spaCy/blob/master/spacy/pipeline/pipes.pyx)
  - `token_vector_width` and `embed_size` are ways of configuring the embedding. We used embeddings in https://colab.research.google.com/notebooks/mlcc/intro_to_sparse_data_and_embeddings.ipynb
- `embed_size`: an environment variable for the number of rows in embedding tables. (https://spacy.io/api/cli#train-hyperparams).

### Evaluation metrics
- Precision, recall, and accuracy we have talked about in class
- F-score (also called F1 score) is the harmonic mean of precision and recall: 2\*(precision\*recall)/(precision+recall). The best possible value is 1 (which would mean perfect precision and perfect recall). This measure assumes precision and recall are equally important, which may not be true for all problems. https://en.wikipedia.org/wiki/F1_score
- macro average vs weighted average for multiclass metrics: weighted average takes into consideration the support (number of examples) for each category, macro average does not. https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

## Implementation

In DATA 303, we used a dataset of Yelp reviews which can be found here (https://www.yelp.com/dataset) (we narrowed it down to restaurant reviews). The work we did is in [EDA](https://rpruim.github.io/ds303/S20/from-class/YELP/yelp_eda.html), [Classifying](https://rpruim.github.io/ds303/S20/from-class/YELP/yelp_classifier.html), and [word vectors](https://rpruim.github.io/ds303/S20/from-class/word-vectors/yelp-word-vectors.html). I am working with that same dataset for my analysis, using a random sample that I saved to my computer so that I would get consistent results. (If you follow the steps in [EDA](https://rpruim.github.io/ds303/S20/from-class/YELP/yelp_eda.html) and save the object `random_reviews` to a csv, you'll get a comparable sample.)

I used the example in spaCy documentation https://github.com/explosion/spaCy/blob/master/examples/training/train_textcat.py to train a `TextCategorizer` and evaluate its performance. I used the example https://www.kaggle.com/poonaml/text-classification-using-spacy to read data into a DataFrame and do the required manipulation to prepare the data for training.

In addition to training a `TextCategorizer` on a new dataset, I'm also adding more code for evaluating performance so that I can see how well the model is performing. I'm using existing libraries where possible (`scikit-learn`, `pandas`, etc.) so as not to reinvent the wheel.

I'm doing binary classification for positive/negative reviews. (Note: following the example in spaCy documentation, I put in both positive and negative categories, so the final output layer is softmax rather than logistic). I'm also doing multiclass classification for predicting star rating (star ratings are 1-5). I haven't seen an example of multiclass classification with `TextCategorizer`, so that's something new I'm adding. As for model architectures, I am doing both `"simple_cnn"` and `"bow"` for each task. (For the `"bow"`, I am using the default `"ngram_size"` of 1.) So that's 4 different models in total to compare.

I have not been using a test set because I haven't changed any hyperparameters so the validation set has been sufficient (I'm just seeing how different models do on the validation set, rather than changing the models to perform better on the validation set).

My implementation, as well as more detailed commentary than what is in the "Results" section of this document, can be found at `yelp_reviews.ipynb`

## Results

It took a while to understand how to use spaCy's `TextCategorizer` and add in evaluation and all of that. But now that I've got things going, I feel it would be relatively easy to put in a new dataset, experiment with hyperparameters, etc. 

It looks like the accuracy scores with spaCy were slightly better than the example we did in DATA 303 class: https://rpruim.github.io/ds303/S20/from-class/YELP/yelp_classifier.html, but since there was a lot to look at just within spaCy itself, I focused most of my analysis on comparison within spaCy rather than on comparing to other classifiers.

In general, the spaCy models start off doing quite well, then get even better on the training data and don't change performance much on the validation set. If I have time I may experiment and see how changing hyperparameters, size of training data, etc. affects these results. There are a few things to keep in mind that may have something to do with why I am seeing these results:

- The data is unbalanced: there are far more positive reviews than negative reviews, and this problem gets worse when we use star categories instead of just binary classification. There is also a slight imbalance in review length: 5-star reviews tend to be shorter (see the violin plot of review lengths in https://rpruim.github.io/ds303/S20/from-class/YELP/yelp_classifier.html). With this imbalance, there's only so well the model can do (so if it starts out good it may not be able to get much better).
- Unlike many other classification models for text classification (e.g., the keras/tensorflow models we did in class/labs), spaCy's models have more information than raw strings. spaCy uses Tokens, which have a lot of information (see https://spacy.io/api/token#attributes). I think spaCy may be using some of this additional information in its models. So spaCy is more specialized to text and thus may do better than a general-purpose Machine Learning technology like Keras.
  - related to this, spaCy's tokenization is nondestructive, meaning that all of the original text (whitespace, punctuation, words, etc.) makes its way into tokens. So the model may be learning non-word information (e.g., maybe 5-star reviews use more exclamation marks) as well.
- the validation set metrics not changing over time may be related to the parameter averaging.

The `"simple_cnn"` performs better than the `"bow"` for binary classification, but in this particular case I'd say it's not enough to make me prefer it over the simpler, faster `"bow"` model. For multiclass classification, the improved performance in `"simple_cnn"` as compared to `"bow"` is more pronounced. The `"bow"` would probably perform better with ngram size larger than 1 though and in that case might get good enough to do almost as well as `"simple_cnn"`.

Note about why bag of words does well: bag of words doesn't care about what words are nearby (further away than ngram-size, that is). It just looks in each document for each ngram that is a feature (which I think in spaCy's case is just ngrams that appears frequently enough) and asks, "how many times does this ngram occur in this document?" That frequency is multiplied by a coefficient and all the terms (frequency\*coefficient for each feature) are summed to get the prediction. All the model has to do is figure out the coefficients, so linear regression is relatively simple and fast. And in the case of reviews, there are certain words that are indicative of certain categories (e.g., "great" for a positive review). The model doesn't have to learn context, just the ngrams that are significant. Classifying a review as positive or negative is a simple enough task (with some clear keywords) that bag-of-words can still do fairly well.

For both multiclass classifiers, performance on 1s and 5s is better than performance on 2-4. While the good performance of 5 can be explained by the prevalence of 5s in the dataset, there are not enough 1s in the dataset for that reasoning to explain why 1s do so well. I think that 1s and 5s tend to use more extreme language and that also contributes to this result (thus there are more obvious keywords and phrases that the model can pick up on).

The patterns in multiclass classification may be relevant for unbalanced multiclass data in general, or they may be more specific to reviews (giving a star-rating to a review is a somewhat subjective process so for example while one person might write a review and rate 4 stars, another person might write the same review and give 5 stars). 

More detailed discussion of the patterns in multiclass classification can be found in `yelp_reviews.ipynb`.

### Areas for future work
- for each of the models, find out which misclassifications was the model most confident on? which misclassifications was the model least confident on? This may provide evidence for or against my hypotheses about why the models performed the way they did on the review data
  - In general, do more of looking at specific texts and seeing how the model performs on them.
- try `"bow"` with a larger ngram size
- try rebalancing the data, probably by undersampling over-represented categories
- try the binary case with only one category (positive) so that the output layer is logistic instead of softmax and see how that affects performance
- try different hyperparameters
- try using a different dataset

## Implications

There is a benefit to being able to use an existing tool and show its usefulness; I'm recognizing and respecting the hard work of the people who have built spaCy. If I were to take this technology to another project that I was working on with other people, I think "here's a production tool that I understand and am able to use" might be an easier sell than "here's something I built from scratch for a class". However, that may not be what you meant by social and ethical implications.

I'm not sure what the social and ethical implications are for the specific task of classifying reviews, since review text is pretty much always accompanied with some kind of review score so it doesn't seem to me like a useful prediction in and of itself.

However, I can think of applications of text categorization that have alarming social/ethical implications. For example, imagine a text categorization system for resumes. Perhaps there are "good" resumes and "bad" resumes, and so some candidates get left out based on textual details that shouldn't be part of the consideration. For example, someone's resume is rejected because they don't have enough academic-sounding words. Or perhaps the model picks up on the fact that many people at a company come from the same school and so prioritizes candidates from that school at the expense of other candidates.

Technologies to flag/categorize resumes already exist, and I find that concerning. What kind of structural injustice are we allowing machines to perpetuate? What kind of new injustices are being introduced as the model selects whatever (potentially difficult to explain/audit) patterns reduce the loss that it was given? Of course resume selection that doesn't involve algorithms has its own issues. The point I want to make is not that technology is bad but rather that we need to use it responsibly. Technology cannot solve racism, sexism, or any form of injustice by itself, and if we pretend we can just put things into an algorithm and have it solve all our problems then we will only perpetuate the same issues for which we just patted ourselves on the back for finding technology to solve.